In [1]:
import pymysql  
import nbimporter   
import TableCreation 

In [2]:
%run TableCreation.ipynb

Connected successfully!
Table 'patients_details' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'treatment' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'difficulty' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'patient_changes' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'pain_symptoms' created successfully!
Tables: (('difficulty_rati

In [3]:
import cv2
import numpy as np
import easyocr
import json
import pytesseract


# Load the image
image_path = "/Users/akhilch/Desktop/example3.png"  # Replace with your image path
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define bounding boxes (field_name: (x, y, width, height))
bounding_boxes = {
    "patient_name": (274, 273, 313, 52),
    "dob": (799, 264, 258, 62),
    "date": (160, 73, 231, 69),
    "injection": (229, 154, 125, 60),
    "exercise_therapy": (1032, 147, 131, 67),

    # Difficulty ratings
    "bending_or_stooping": (324, 400, 153, 59),
    "Putting_on_shoes": (284, 462, 158, 60),
    "Sleeping": (188, 525, 143, 49),
    "Standing_for_an_hour": (332, 582, 141, 50),
    "Going_up_or_down_a_flight_of_stairs": (485, 636, 143, 65),
    "Walking_through_a_store": (370, 699, 150, 62),
    "Driving_for_an_hour": (305, 755, 156, 60),
    "Preparing_a_meal": (287, 820, 149, 61),
    "Yard_work": (202, 885, 149, 55),
    "Picking_up_items_off_the_floor": (429, 942, 147, 58),

    # Patient changes
    "since_last_treatment": (93, 1054, 316, 74),
    "since_start_of_treatment": (104, 1172, 420, 74),
    "last_3_days": (114, 1291, 394, 77),

    # Pain symptoms
    "pain": (146, 1415, 78, 50),
    "numbness": (374, 1411, 78, 53),
    "tingling": (569, 1412, 88, 54),
    "burning": (783, 1415, 79, 54),
    "tightness": (1007, 1408, 61, 59),

    # Medical assistant data
    "blood_pressure": (289, 1533, 101, 59),
    "hr": (464, 1537, 77, 48),
    "weight": (675, 1533, 110, 56),
    "height": (893, 1532, 104, 57),
    "spo2": (174, 1656, 73, 43),
    "temperature": (439, 1657, 67, 41),
    "blood_glucose": (719, 1655, 82, 47),
    "respirations": (961, 1660, 73, 41),

    # Program details
    "program_number": (305, 1604, 68, 45),
    "treatment_number": (631, 1601, 65, 44),
    "placement": (854, 1600, 204, 36),
}

# Initialize OCR reader
reader = easyocr.Reader(['en'])

# Dictionary to store results
output_data = {
    "patient_name": "",
    "dob": "",
    "date": "",
    "injection": "",
    "exercise_therapy": "",
    "difficulty_ratings": {},
    "patient_changes": {},
    "pain_symptoms": {},
    "medical_assistant_data": {},
    "program_details": {}
}

# Extract text for standard fields
standard_fields = [
    "patient_name", "dob", "date", "since_last_treatment",
    "since_start_of_treatment", "last_3_days", "injection", "exercise_therapy",
    "blood_pressure", "hr", "weight", "height", "spo2", "temperature",
    "blood_glucose", "respirations", "program_number", "treatment_number", "placement"
]

for field in standard_fields:
    x, y, w, h = bounding_boxes[field]
    roi = gray[y:y+h, x:x+w]
    
    # Run OCR on the extracted region
    result = reader.readtext(roi, detail=0)
    
    if result:
        # Assign to correct nested dictionary if applicable
        if field in ["since_last_treatment", "since_start_of_treatment", "last_3_days"]:
            output_data["patient_changes"][field] = result[0]
        elif field in ["blood_pressure", "hr", "weight", "height", "spo2", "temperature", "blood_glucose", "respirations"]:
            output_data["medical_assistant_data"][field] = result[0]
        elif field in ["program_number", "treatment_number", "placement"]:
            output_data["program_details"][field] = result[0]
        else:
            output_data[field] = result[0]
    else:
        output_data[field] = "N/A"

# Extract circled numbers for difficulty ratings & pain symptoms
circled_fields = {
    "difficulty_ratings": ["bending_or_stooping", "Putting_on_shoes", "Sleeping",
        "Standing_for_an_hour", "Going_up_or_down_a_flight_of_stairs",
        "Walking_through_a_store", "Driving_for_an_hour",
        "Preparing_a_meal", "Yard_work", "Picking_up_items_off_the_floor"
    ],
    "pain_symptoms": [
        "pain", "numbness", "tingling", "burning", "tightness"
    ]
}

for category, fields in circled_fields.items():
    for field in fields:
        x, y, w, h = bounding_boxes[field]
        roi = gray[y:y+h, x:x+w]

        # Apply thresholding to enhance visibility
        _, binary = cv2.threshold(roi, 150, 255, cv2.THRESH_BINARY_INV)

        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        detected_number = "?"  # Default value if no number is found

        for contour in contours:
            # Get the minimum enclosing circle
            (cx, cy), radius = cv2.minEnclosingCircle(contour)
            cx, cy, radius = int(cx), int(cy), int(radius)

            # Filter small or large circles
            if 14 < radius < 25:
                x1, y1 = max(0, cx - radius), max(0, cy - radius)
                x2, y2 = min(w, cx + radius), min(h, cy + radius)

                circled_roi = roi[y1:y2, x1:x2]

                if circled_roi.shape[0] > 5 and circled_roi.shape[1] > 5:
                    _, thresholded = cv2.threshold(circled_roi, 150, 255, cv2.THRESH_BINARY_INV)

                    # Run OCR to extract number
                    text = pytesseract.image_to_string(thresholded, config="--psm 10 -c tessedit_char_whitelist=0123456789")
                    detected_number = text.strip() if text.strip().isdigit() else "?"

                    cv2.circle(image, (x+cx, y+cy), radius, (0, 255, 0), 2)
                    cv2.putText(image, detected_number, (x+cx-10, y+cy-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        output_data[category][field] = detected_number

# Save output as JSON
with open("output.json", "w") as json_file:
    json.dump(output_data, json_file, indent=4)

##print(json.dumps(output_data, indent=4))

# Convert extracted data to JSON format
json_output = json.dumps(output_data, indent=4)

# Save JSON output to a file
with open("output.json", "w") as json_file:
    json_file.write(json_output)

json_output = json.loads(json_output)
# Print JSON output
print(json_output)

{'patient_name': '8aVi', 'dob': '13-#-95', 'date': '-5-Q4', 'injection': 'YES', 'exercise_therapy': 'YES (NO', 'difficulty_ratings': {'bending_or_stooping': '0', 'Putting_on_shoes': '45', 'Sleeping': '2', 'Standing_for_an_hour': '2', 'Going_up_or_down_a_flight_of_stairs': '4', 'Walking_through_a_store': '?', 'Driving_for_an_hour': '0', 'Preparing_a_meal': '4', 'Yard_work': '1', 'Picking_up_items_off_the_floor': '16'}, 'patient_changes': {'since_last_treatment': 'Good', 'since_start_of_treatment': 'bad', 'last_3_days': 'Wo { st'}, 'pain_symptoms': {'pain': '?', 'numbness': '?', 'tingling': '4', 'burning': '?', 'tightness': '?'}, 'medical_assistant_data': {}, 'program_details': {}, 'blood_pressure': 'N/A', 'hr': 'N/A', 'weight': 'N/A', 'height': 'N/A', 'spo2': 'N/A', 'temperature': 'N/A', 'blood_glucose': 'N/A', 'respirations': 'N/A', 'program_number': 'N/A', 'treatment_number': 'N/A', 'placement': 'N/A'}


In [4]:


# Properly formatted JSON output
print(json.dumps(output_data, indent=4))


{
    "patient_name": "8aVi",
    "dob": "13-#-95",
    "date": "-5-Q4",
    "injection": "YES",
    "exercise_therapy": "YES (NO",
    "difficulty_ratings": {
        "bending_or_stooping": "0",
        "Putting_on_shoes": "45",
        "Sleeping": "2",
        "Standing_for_an_hour": "2",
        "Going_up_or_down_a_flight_of_stairs": "4",
        "Walking_through_a_store": "?",
        "Driving_for_an_hour": "0",
        "Preparing_a_meal": "4",
        "Yard_work": "1",
        "Picking_up_items_off_the_floor": "16"
    },
    "patient_changes": {
        "since_last_treatment": "Good",
        "since_start_of_treatment": "bad",
        "last_3_days": "Wo { st"
    },
    "pain_symptoms": {
        "pain": "?",
        "numbness": "?",
        "tingling": "4",
        "burning": "?",
        "tightness": "?"
    },
    "medical_assistant_data": {},
    "program_details": {},
    "blood_pressure": "N/A",
    "hr": "N/A",
    "weight": "N/A",
    "height": "N/A",
    "spo2": "N/A",
 

In [5]:
# Assuming json_output is a dictionary containing patient_name and dob
query = """INSERT INTO patient_details (patient_name, dob) 
           VALUES (%s, %s)"""

data = (json_output["patient_name"], json_output["dob"])

try:
    with connection.cursor() as cursor:
        cursor.execute(query, data)  # Safe execution
        patient_id = cursor.lastrowid
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")

✅ Data inserted successfully!


In [6]:
treatment_details = (patient_id,json_output["date"],json_output["injection"],json_output["exercise_therapy"])
treatment_query = """INSERT INTO treatment_details (patient_id, date,injection,exercise_therapy) 
           VALUES (%s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(treatment_query, treatment_details)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(22, '-5-Q4', 'YES', 'YES (NO')


In [7]:
print(treatment_details)

(22, '-5-Q4', 'YES', 'YES (NO')


In [12]:


difficulty_ratings = (patient_id ,json_output["date"],json_output["difficulty_ratings"]["bending_or_stooping"],json_output["difficulty_ratings"]["Putting_on_shoes"],json_output["difficulty_ratings"]["Sleeping"])


difficulty_query = """INSERT INTO difficulty_ratings (patient_id,date,bending_or_stooping,putting_on_shoes,sleeping) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(difficulty_query,difficulty_ratings)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(22, '-5-Q4', 'YES', 'YES (NO')


In [13]:
print(difficulty_ratings)

(22, '-5-Q4', '0', '45', '2')


In [15]:


patient_changes = (patient_id ,json_output["date"],json_output["patient_changes"]["since_last_treatment"],json_output["patient_changes"]["since_start_of_treatment"],json_output["patient_changes"]["last_3_days"])


patient_query = """INSERT INTO patient_changes(patient_id, date,since_last_treatment,since_start_of_treatment,last_3_days) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(patient_query,patient_changes)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(22, '-5-Q4', 'YES', 'YES (NO')


In [17]:


pain_symptoms= (patient_id ,json_output["date"],json_output["pain_symptoms"]["pain"],json_output["pain_symptoms"]["numbness"],json_output["pain_symptoms"]["tingling"],
                json_output["pain_symptoms"]["burning"],json_output["pain_symptoms"]["tightness"])


pain_query = """INSERT INTO pain_symptoms(patient_id, date,pain,numbness,tingling,burning,tightness) 
           VALUES (%s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(pain_query,pain_symptoms)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(22, '-5-Q4', 'YES', 'YES (NO')


In [18]:


medical_assistant_data= (patient_id ,json_output["date"],json_output["blood_pressure"],json_output["hr"],json_output["weight"],json_output["height"],json_output["spo2"],json_output["temperature"],
                         json_output["blood_glucose"],json_output["respirations"])


medical_query = """INSERT INTO medical_assistant_data(patient_id, date,blood_pressure,hr,weight,height,spo2,temperature,blood_glucose,respirations) 
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(medical_query,medical_assistant_data)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(22, '-5-Q4', 'YES', 'YES (NO')
